In [11]:
import angr
import archinfo
arch = archinfo.ArchAArch64()

In [12]:
# Loading the elf fila and creating a project
path_to_binary = './tee.elf'
proj = angr.Project(path_to_binary, auto_load_libs=False)

In [13]:
cfg = proj.analyses.CFG()

WARNING  | 2024-05-13 22:37:31,596 | claripy.vsa.strided_interval | Tried to cast_low an interval to an interval shorter than its stride.
WARNING  | 2024-05-13 22:37:37,237 | claripy.vsa.strided_interval | Tried to cast_low an interval to an interval shorter than its stride.


In [14]:
# Creating a initial state and simulation manager to perform symbolic execution on the particular function.
function_name = "syscall_open_ta_session"
function_address = proj.loader.find_symbol(function_name).rebased_addr
initial_state = proj.factory.blank_state(addr=function_address)
simgr = proj.factory.simulation_manager(initial_state, save_unconstrained=True)

# Defining functions to perform backtracking
def get_node_from_address(address):
    all_nodes = list(cfg.graph.nodes)
    for node in all_nodes:
        if node.addr == address:
            return node
    return None

def handle_spin_to_backtrack(LocalLoopSeer, succ_state):
    LocalLoopSeer.cut_succs.append(succ_state)

# Exploring with Breadth First Search (Default) and Local flag Seer to detect loops
simgr.use_technique(angr.exploration_techniques.LocalLoopSeer(bound=1, bound_reached=handle_spin_to_backtrack))

In [ ]:
# Exploring with Depth First Search (Optional)
from angr.exploration_techniques import DFS
simgr.use_technique(DFS())

In [15]:
# Exploration Handler Functions
def get_function_name(project, current_pc):
    try:
        function = cfg.functions[current_pc]

    except (KeyError, AttributeError) as e:
        pass
    
def isValid(addr):
    try:
        proj.factory.block(int(addr, 16))
        return True
    except:
        return False

In [16]:
# Initializing Variables
count = 0
level = 1
first_iter = True
branch_address_1 = None
branch_address_2 = None
branch_addresses = []
flag = True

In [10]:
while flag:
    for state in simgr.active:

        print('Simgr.active - ', simgr.active)
        print("Level: ", level)
        level += 1

        branch_address_2 = None
        branch_address_1 = None
        # Get the current memory address as an integer
        current_pc = state.regs.pc

        try:
            # Convert BitVector to integer for printing
            current_pc_int = current_pc.args[0]
            print(f"Current Address: {hex(current_pc_int)}")
        except Exception as e:
            pass
            # print(f"Error converting PC to integer: {e}")
        current_block = proj.factory.block(state.addr)
        print("Current block instructions:")
        branch_insn = None
        for insn in current_block.capstone.insns:
            if insn == None:
                break
            branch_insn = insn
            print(f"{insn.mnemonic} {insn.op_str}")
        if branch_insn == None:
            continue
        branch_address_1 = branch_insn.address+4
        branch_addresses.append(branch_address_1)
        branch_address_2 = branch_insn.op_str.split(",")[-1][1:]
        try:
            if isValid(branch_address_2):
                branch_address_2 = int(branch_address_2, 16)
                branch_addresses.append(branch_address_2)
        except ValueError:
            pass
        get_function_name(proj, current_pc_int)
        print("-----------------------------")
    if flag == False:
        break
    print(branch_addresses)
    simgr.step()
    for b_addr in branch_addresses:
        add = True
        for state in simgr.active:
            if b_addr == state.addr:
                add = False
        if add:
            count = count + 1
            state = proj.factory.blank_state(addr=b_addr) # inherit parent properties here
            simgr.active.append(state)
        else:
            branch_addresses.remove(b_addr)

    if not first_iter:
        while count > 0:
            pop_addr = branch_addresses.pop(0)
            for state in simgr.active:
                if state.addr == pop_addr:
                    simgr.active.remove(state)
            count = count - 1
    first_iter = False
    
    if level > 100:
        break

WARNING  | 2024-05-13 22:30:35,394 | angr.storage.memory_mixins.default_filler_mixin | Filling register x29 with 8 unconstrained bytes referenced from 0xe12a40c (syscall_open_ta_session+0x0 in tee.elf (0xe12a40c))
WARNING  | 2024-05-13 22:30:35,396 | angr.storage.memory_mixins.default_filler_mixin | Filling register x30 with 8 unconstrained bytes referenced from 0xe12a40c (syscall_open_ta_session+0x0 in tee.elf (0xe12a40c))
WARNING  | 2024-05-13 22:30:35,398 | angr.storage.memory_mixins.default_filler_mixin | Filling register x21 with 8 unconstrained bytes referenced from 0xe12a414 (syscall_open_ta_session+0x8 in tee.elf (0xe12a414))
WARNING  | 2024-05-13 22:30:35,400 | angr.storage.memory_mixins.default_filler_mixin | Filling register x22 with 8 unconstrained bytes referenced from 0xe12a414 (syscall_open_ta_session+0x8 in tee.elf (0xe12a414))
WARNING  | 2024-05-13 22:30:35,401 | angr.storage.memory_mixins.default_filler_mixin | Filling register x2 with 8 unconstrained bytes referenced

Level:  1
[236102740, 236027460]
Level:  2
Level:  3
[236102740, 236027472, 236027436, 236102752, 236100852]


WARNING  | 2024-05-13 22:30:35,597 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x12 with 8 unconstrained bytes referenced from 0xe12a458 (syscall_open_ta_session+0x4c in tee.elf (0xe12a458))
WARNING  | 2024-05-13 22:30:35,617 | angr.storage.memory_mixins.default_filler_mixin | Filling register x29 with 8 unconstrained bytes referenced from 0xe129cf4 (to_user_ta_ctx+0x0 in tee.elf (0xe129cf4))
WARNING  | 2024-05-13 22:30:35,620 | angr.storage.memory_mixins.default_filler_mixin | Filling register x19 with 8 unconstrained bytes referenced from 0xe129cfc (to_user_ta_ctx+0x8 in tee.elf (0xe129cfc))


Level:  4
Level:  5
[236027448, 236039548, 236100872, 236040648]
Level:  6
Level:  7
[236100872, 236039560, 236039144, 236040652]
Level:  8
Level:  9
Level:  10
[236040652, 236039156, 236039024, 236040684, 236040672, 236040676]


WARNING  | 2024-05-13 22:30:35,836 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0xfffffffffffffff0 with 8 unconstrained bytes referenced from 0xe11b1cc (is_user_ta_ctx+0x4 in tee.elf (0xe11b1cc))
WARNING  | 2024-05-13 22:30:35,966 | angr.storage.memory_mixins.default_filler_mixin | Filling register x20 with 8 unconstrained bytes referenced from 0xe15cdd8 (malloc+0x8 in tee.elf (0xe15cdd8))
WARNING  | 2024-05-13 22:30:35,967 | angr.storage.memory_mixins.default_filler_mixin | Filling register x21 with 8 unconstrained bytes referenced from 0xe15cde8 (malloc+0x18 in tee.elf (0xe15cde8))
WARNING  | 2024-05-13 22:30:35,971 | angr.storage.memory_mixins.default_filler_mixin | Filling register x20 with 8 unconstrained bytes referenced from 0xe15ebd4 (trace_printf+0x8 in tee.elf (0xe15ebd4))
WARNING  | 2024-05-13 22:30:35,972 | angr.storage.memory_mixins.default_filler_mixin | Filling register x21 with 8 unconstrained bytes referenced from 0xe15ebe4 (trace_printf+0x18 i

...

Level:  57
Level:  58
Level:  59
Level:  60
Level:  61
Level:  62
Level:  63
Level:  64
Level:  65
Level:  66
Level:  67
Level:  68
[235948608, 235948608, 236021336, 236021400, 236317760, 235948440, 236021476, 236037036, 235948428, 235948680, 235948680, 236021336, 236021336, 236021400, 236317644, 236317760, 235944128, 235944136, 235944328, 235948464, 236021400, 236317644, 235948612, 236003680, 236021488]


WARNING  | 2024-05-13 22:30:36,680 | angr.engines.successors | Exit state has over 256 possible solutions. Likely unconstrained; skipping. <BV64 reg_x3_1545_64{UNINITIALIZED}>
WARNING  | 2024-05-13 22:30:36,744 | angr.storage.memory_mixins.default_filler_mixin | Filling register x29 with 8 unconstrained bytes referenced from 0xe15ebcc (trace_printf+0x0 in tee.elf (0xe15ebcc))
WARNING  | 2024-05-13 22:30:36,748 | angr.storage.memory_mixins.default_filler_mixin | Filling register x23 with 8 unconstrained bytes referenced from 0xe15ebec (trace_printf+0x20 in tee.elf (0xe15ebec))
WARNING  | 2024-05-13 22:30:36,749 | angr.storage.memory_mixins.default_filler_mixin | Filling register x24 with 8 unconstrained bytes referenced from 0xe15ebec (trace_printf+0x20 in tee.elf (0xe15ebec))
WARNING  | 2024-05-13 22:30:36,752 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x1c2881c0 with 8 unconstrained bytes referenced from 0xe15ec0c (trace_printf+0x40 in tee.elf (0xe15ec0c))
W

Level:  69
Level:  70
Level:  71
Level:  72
Level:  73
Level:  74
Level:  75
Level:  76
Level:  77
Level:  78
Level:  79
Level:  80
Level:  81
Level:  82
[236021336, 236021400, 236317760, 235944128, 235944328, 236021400, 236317644, 235948612, 236021488, 235948440, 235948728, 235948728, 236021476, 236021352, 236021476, 236021352, 236317760, 235944128, 235944136, 235944328, 235944148, 235944344, 235948500, 235936072, 235948496, 235947912, 236317760, 235944128, 236003692, 235948428, 236021404, 236021236, 236317788, 236317012]
Level:  83
Level:  84
Level:  85
Level:  86
Level:  87
Level:  88
Level:  89
Level:  90
Level:  91
Level:  92
Level:  93
Level:  94
Level:  95
Level:  96
Level:  97
[236021476, 236021476, 236317760, 235944136, 235944328, 235944344, 235948500, 235948496, 236317760, 236003692, 236021404, 236317788, 236037040, 236307416, 235948428, 236021208, 236021400, 236317644, 236021400, 236317644, 235944136, 235944328, 235944148, 235944344, 235944224, 235944176, 235936080, 23593608

WARNING  | 2024-05-13 22:30:36,961 | angr.storage.memory_mixins.default_filler_mixin | Filling register x0 with 8 unconstrained bytes referenced from 0xe1049d4 (select_vector_wa_spectre_bhb+0x3c in tee.elf (0xe1049d4))
WARNING  | 2024-05-13 22:30:36,964 | angr.storage.memory_mixins.default_filler_mixin | Filling register x19 with 8 unconstrained bytes referenced from 0xe1049e4 (select_vector_wa_spectre_bhb+0x4c in tee.elf (0xe1049e4))
WARNING  | 2024-05-13 22:30:37,161 | angr.storage.memory_mixins.default_filler_mixin | Filling register x29 with 8 unconstrained bytes referenced from 0xe15ebcc (trace_printf+0x0 in tee.elf (0xe15ebcc))
WARNING  | 2024-05-13 22:30:37,165 | angr.storage.memory_mixins.default_filler_mixin | Filling register x23 with 8 unconstrained bytes referenced from 0xe15ebec (trace_printf+0x20 in tee.elf (0xe15ebec))
WARNING  | 2024-05-13 22:30:37,166 | angr.storage.memory_mixins.default_filler_mixin | Filling register x24 with 8 unconstrained bytes referenced from 0xe

Level:  98
Level:  99
Level:  100
Level:  101
Level:  102
Level:  103
Level:  104
Level:  105
Level:  106
Level:  107
Level:  108
Level:  109
Level:  110
Level:  111
Level:  112
Level:  113
Level:  114
Level:  115
Level:  116
Level:  117
[236021400, 236021400, 235944136, 235944328, 235944344, 235944176, 235936080, 235947924, 235944136, 235944328, 235948440, 236021240, 236317092, 235948524, 236037072, 236036884, 235948440, 236021300, 236317760, 235944128, 236317760, 235944128, 235944148, 235944344, 235944224, 235944176, 235944240, 235944356, 235936108, 236003788, 236317644, 235944148, 235944344, 236003696, 235963024, 236317644, 236317516, 236317128, 236317108, 236317480, 236038496, 236037024, 236021404, 236021236, 235944340, 235947928, 236003800, 236021244, 236012544]


In [ ]:
vars(simgr)

In [ ]:
print(flag)